In [18]:
import pandas as pd
import os
import json
from dotenv import load_dotenv

load_dotenv()

True

# A Graph RAG Approach to Query-Focused Summarization
<hr>


Reference: [From Local to Global: A Graph RAG Approach to Query-Focused Summarization](https://arxiv.org/abs/2404.16130)
<br>

![image](../.resources/Query-Focused-Summarization.png)


**Benchmark Dataset**

The benchmark dataset to test RAG application is [MultiHop-RAG](https://arxiv.org/abs/2401.15391): a QA dataset to evaluate retrieval and reasoning across documents with metadata in the RAG pipelines. It contains 2556 queries, with evidence for each query distributed across 2 to 4 documents. The queries also involve document metadata, reflecting complex scenarios commonly found in real-world RAG applications.

In [19]:
df = pd.read_json("hf://datasets/yixuantt/MultiHopRAG/MultiHopRAG.json")
df.head()

,query,answer,question_type,evidence_list
0,Who is the individual associated with the cryp...,Sam Bankman-Fried,inference_query,[{'title': 'The FTX trial is bigger than Sam B...
1,Which individual is implicated in both inflati...,Donald Trump,inference_query,[{'title': 'Donald Trump defrauded banks with ...
2,Who is the figure associated with generative A...,Sam Altman,inference_query,[{'title': 'OpenAI's ex-chairman accuses board...
3,Do the TechCrunch article on software companie...,Yes,comparison_query,"[{'title': 'Here’s how Rainforest, a budding S..."
4,Which online betting platform provides a welco...,Caesars Sportsbook,inference_query,[{'title': '2023 Kentucky online sports bettin...


In [20]:
df.head().to_dict('records')[0]

{'query': 'Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud and conspiracy charges, as reported by both The Verge and TechCrunch, and is accused by prosecutors of committing fraud for personal gain?',
 'answer': 'Sam Bankman-Fried',
 'question_type': 'inference_query',
 'evidence_list': [{'title': 'The FTX trial is bigger than Sam Bankman-Fried',
   'author': 'Elizabeth Lopatto',
   'url': 'https://www.theverge.com/2023/9/28/23893269/ftx-sam-bankman-fried-trial-evidence-crypto',
   'source': 'The Verge',
   'category': 'technology',
   'published_at': '2023-09-28T12:00:00+00:00',
   'fact': 'Before his fall, Bankman-Fried made himself out to be the Good Boy of crypto — the trustworthy face of a sometimes-shady industry.'},
  {'title': 'SBF’s trial starts soon, but how did he — and FTX — get here?',
   'author': 'Jacquelyn Melinek',
   'url': 'https://techcrunch.com/2023/10/01/ftx-lawsuit-timeline/',
   'source': 'TechCrunch',
   'catego

In [21]:
# Save dataset for later use
with open("../json/multi_hop_rag.json", "w") as fp:
    json.dump(df.to_dict('records'), fp)

<br>
<br>
<hr>

## 1 Source Document -> Text Chunks

The chunk size used for this implementation is 600 tokens. According to the paper, using a longer chunk size can reduce the number of LLM calls required for extraction. However, this approach can lead to recall degradation, as the model may not fully extract all relevant information from a longer context window.

In dataflow from [Microsoft GraphRAG](https://microsoft.github.io/graphrag/posts/index/1-default_dataflow/), this state is convered in **Phrase 1: Compose TextUnits**

![image](../.resources/grag_phrase1.png)

In [22]:
# load data
data = ""

with open("../doc/pg24022.txt", 'r') as file:
    data = '\n'.join(file.readlines())

print(data[:1200])

The Project Gutenberg eBook of A Christmas Carol

    

This ebook is for the use of anyone anywhere in the United States and

most other parts of the world at no cost and with almost no restrictions

whatsoever. You may copy it, give it away or re-use it under the terms

of the Project Gutenberg License included with this ebook or online

at www.gutenberg.org. If you are not located in the United States,

you will have to check the laws of the country where you are located

before using this eBook.



Title: A Christmas Carol



Author: Charles Dickens



Illustrator: Arthur Rackham



Release date: December 24, 2007 [eBook #24022]



Language: English



Original publication: Philadelphia and New York: J. B. Lippincott Company,, 1915



Credits: Produced by Suzanne Shell, Janet Blenkinship and the Online

        Distributed Proofreading Team at http://www.pgdp.net





*** START OF THE PROJECT GUTENBERG EBOOK A CHRISTMAS CAROL ***









Produced by Suzanne Shell, Janet Blenkinshi

In [23]:
from text_splitter import TextSplitter
text_splitter = TextSplitter(600, 20)

chunks = text_splitter.split_text(data)
chunks[:5]

['\ufeffThe Project Gutenberg eBook of A Christmas Carol\n\n    \n\nThis ebook is for the use of anyone anywhere in the United States and\n\nmost other parts of the world at no cost and with almost no restrictions\n\nwhatsoever. You may copy it, give it away or re-use it under the terms\n\nof the Project Gutenberg License included with this ebook or online\n\nat www.gutenberg.org. If you are not located in the United States,\n\nyou will have to check the laws of the country where you are located\n\nbefore using this eBook.\n\n\n\nTitle: A Christmas Carol\n\n\n\nAuthor: Charles Dickens\n\n\n\nIllustrator: Arthur Rackham\n\n\n\nRelease date: December 24, 2007 [eBook #24022]\n\n\n\nLanguage: English\n\n\n\nOriginal publication: Philadelphia and New York: J. B. Lippincott Company,, 1915\n\n\n\nCredits: Produced by Suzanne Shell, Janet Blenkinship and the Online\n\n        Distributed Proofreading Team at http://www.pgdp.net\n\n\n\n\n\n*** START OF THE PROJECT GUTENBERG EBOOK A CHRISTMAS CA

<hr>

## 2 Text extraction -> Element Instances

A multipart LLM prompt will be used to identify all the *entites* in the text, including their names, type and description. Then extracting *relationship* between entities

In [24]:
from extractor import GraphExtractor
from LLM import GeminiModel

ge = GraphExtractor(GeminiModel())

In [30]:
ge.extract_text(chunks[0], store=False)

"entity"<TD>CHARLES DICKENS<TD>PERSON<TD>Charles Dickens wrote A Christmas Carol
"entity"<TD>BOB CRATCHIT<TD>PERSON<TD>Bob Cratchit is a clerk that works for Ebenezer Scrooge
"entity"<TD>PETER CRATCHIT<TD>PERSON<TD>Peter Cratchit is a son of Bob Cratchit
"entity"<TD>TIM CRATCHIT<TD>PERSON<TD>Tim Cratchit is the youngest son of Bob Cratchit and is refered to as Tiny Tim
"entity"<TD>Ebenezer Scrooge<TD>PERSON<TD>Ebenezer Scrooge is the main character in A Christmas Carol)
<COMPLETE>


[{'entity': {'entity_name': 'CHARLES DICKENS',
   'entity_type': 'PERSON',
   'entity_description': 'Charles Dickens wrote A Christmas Carol'}},
 {'entity': {'entity_name': 'BOB CRATCHIT',
   'entity_type': 'PERSON',
   'entity_description': 'Bob Cratchit is a clerk that works for Ebenezer Scrooge'}},
 {'entity': {'entity_name': 'PETER CRATCHIT',
   'entity_type': 'PERSON',
   'entity_description': 'Peter Cratchit is a son of Bob Cratchit'}},
 {'entity': {'entity_name': 'TIM CRATCHIT',
   'entity_type': 'PERSON',
   'entity_description': 'Tim Cratchit is the youngest son of Bob Cratchit and is refered to as Tiny Tim'}},
 {'entity': {'entity_name': 'Ebenezer Scrooge',
   'entity_type': 'PERSON',
   'entity_description': 'Ebenezer Scrooge is the main character in A Christmas Carol)\n<COMPLETE>'}}]

From the above *entities* and *relationships* extraction using Gemini model. The model is capable of extracting and building information for entities and relationship. 

However, through many tests, the results are not consistent. Sometimes, the model fails to fully extract entities or relationships from text. This has been mentioned in paper, where the author use multiple rounds of “gleanings” to encourage the LLM to detect any additional entities it may have missed on prior extraction rounds.  

**NOTE**: Currently, the function has not included the aboved multi-gleaning process.


<hr>

## 3 Element Instances -> Element Summaries

In second phrase of dataflow from [Microsoft GraphRAG](https://microsoft.github.io/graphrag/posts/index/1-default_dataflow/). After retrieving all the *entities* and *relationships* from list of chunks, we will merge a the entity with the same *name* and *type*, generate list of *description*. This also applies for *relationship* by merging any relationship that have the same pair of *source* and *target*.

After that, for every merged node, a LLM will be used to summarize list of description into a single description, covers full information of the node.